In [6]:
import pandas as pd
import re

import geopandas as gpd


In [8]:
raw = pd.read_csv('data/solo_1인가구 통계.csv', encoding='cp949')
raw.head()

,행정구역별(시군구),연령별,항목,단위,2015 년,2016 년,2017 년,2018 년,2019 년,2020 년,2021 년,2022 년,Unnamed: 12
0,서울특별시,합계,1인가구,NaN,1115744,1138860,1180540,1229421,1299787,1390701,1489893,1564187,NaN
1,서울특별시,합계,남자,NaN,544059,544607,561524,581944,611561,650999,698324,730639,NaN
2,서울특별시,합계,여자,NaN,571685,594253,619016,647477,688226,739702,791569,833548,NaN
3,서울특별시,20세 미만,1인가구,NaN,13721,14937,14190,14487,12806,14185,10687,12872,NaN
4,서울특별시,20세 미만,남자,NaN,5604,6174,5769,5629,5010,5345,4021,5074,NaN


In [9]:
pop = raw.copy()
pop.drop(columns=['단위', 'Unnamed: 12'], inplace=True)
pop.head()

,행정구역별(시군구),연령별,항목,2015 년,2016 년,2017 년,2018 년,2019 년,2020 년,2021 년,2022 년
0,서울특별시,합계,1인가구,1115744,1138860,1180540,1229421,1299787,1390701,1489893,1564187
1,서울특별시,합계,남자,544059,544607,561524,581944,611561,650999,698324,730639
2,서울특별시,합계,여자,571685,594253,619016,647477,688226,739702,791569,833548
3,서울특별시,20세 미만,1인가구,13721,14937,14190,14487,12806,14185,10687,12872
4,서울특별시,20세 미만,남자,5604,6174,5769,5629,5010,5345,4021,5074


In [11]:
year_group = [x for x in range(2015, 2023)]

pop.columns = ['시군구', '연령', '항목'] + year_group
pop.head()

,시군구,연령,항목,2015,2016,2017,2018,2019,2020,2021,2022
0,서울특별시,합계,1인가구,1115744,1138860,1180540,1229421,1299787,1390701,1489893,1564187
1,서울특별시,합계,남자,544059,544607,561524,581944,611561,650999,698324,730639
2,서울특별시,합계,여자,571685,594253,619016,647477,688226,739702,791569,833548
3,서울특별시,20세 미만,1인가구,13721,14937,14190,14487,12806,14185,10687,12872
4,서울특별시,20세 미만,남자,5604,6174,5769,5629,5010,5345,4021,5074


In [ ]:
sido_list = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원특별자치도',
              '충청북도', '충청남도', '전북특별자치도', '전라남도', '경상북도', '경상남도', '제주특별자치도'] 

In [23]:
def convert_age_group(age_str):
    if age_str == '20세 미만':
        return '10'
    elif age_str == '85세 이상':
        return '85'
    else:
        # 20~24 -> 20
        return age_str.split('~')[0]

array(['서울특별시', '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구',
       '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구',
       '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '부산광역시',
       '읍부', '면부', '동부', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구',
       '해운대구', '사하구', '금정구', '연제구', '수영구', '사상구', '기장군', '대구광역시', '수성구',
       '달서구', '달성군', '인천광역시', '연수구', '남동구', '부평구', '계양구', '미추홀구', '강화군',
       '옹진군', '광주광역시', '광산구', '대전광역시', '유성구', '대덕구', '울산광역시', '울주군',
       '세종특별자치시', '세종시', '경기도', '수원시', '장안구', '권선구', '팔달구', '영통구', '성남시',
       '수정구', '중원구', '분당구', '의정부시', '안양시', '만안구', '동안구', '부천시', '원미구',
       '소사구', '오정구', '광명시', '평택시', '동두천시', '안산시', '상록구', '단원구', '고양시',
       '덕양구', '일산동구', '일산서구', '과천시', '구리시', '남양주시', '오산시', '시흥시', '군포시',
       '의왕시', '하남시', '용인시', '처인구', '기흥구', '수지구', '파주시', '이천시', '안성시',
       '김포시', '화성시', '광주시', '양주시', '포천시', '여주시', '연천군', '가평군', '양평군',
       '강원도', '춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시', '삼척시', '홍천군',

In [37]:
pop['연령'] = pop['연령'].apply(convert_age_group)

In [39]:
pop1 = pop.groupby(['시군구', '연령', '항목'])[year_group].agg('sum')
pop1.reset_index(inplace=True)
pop1

,시군구,연령,항목,2015,2016,2017,2018,2019,2020,2021,2022
0,가평군,10,1인가구,17,20,14,12,13,18,14,16
1,가평군,10,남자,9,12,8,9,9,8,6,9
2,가평군,10,여자,8,8,6,X,X,10,8,7
3,가평군,20,1인가구,204,205,170,177,152,184,263,212
4,가평군,20,남자,144,140,101,127,92,102,153,124
...,...,...,...,...,...,...,...,...,...,...,...
12475,흥덕구,85,남자,61,66,72,94,112,123,147,159
12476,흥덕구,85,여자,238,270,329,362,426,475,595,664
12477,흥덕구,합계,1인가구,27819,29523,31670,32647,35043,39230,43153,44750
12478,흥덕구,합계,남자,14688,15469,16975,17792,18894,20935,23226,24204


In [33]:
pop1


,시군구,연령,항목,2015,2016,2017,2018,2019,2020,2021,2022,연령대
0,가평군,20~24,1인가구,204,205,170,177,152,184,263,212,20
1,가평군,20~24,남자,144,140,101,127,92,102,153,124,20
2,가평군,20~24,여자,60,65,69,50,60,82,110,88,20
3,가평군,20세 미만,1인가구,17,20,14,12,13,18,14,16,10
4,가평군,20세 미만,남자,9,12,8,9,9,8,6,9,10
...,...,...,...,...,...,...,...,...,...,...,...,...
12475,흥덕구,85세 이상,남자,61,66,72,94,112,123,147,159,85
12476,흥덕구,85세 이상,여자,238,270,329,362,426,475,595,664,85
12477,흥덕구,합계,1인가구,27819,29523,31670,32647,35043,39230,43153,44750,합계
12478,흥덕구,합계,남자,14688,15469,16975,17792,18894,20935,23226,24204,합계


In [30]:
data = {'연령': ['20세 미만', '20~24', '25~29', '30~34', '35~39', '40~44', '45~49', '50~54', '55~59', '60~64', '65~69', '70~74', '75~79', '80~84', '85세 이상']}
df = pd.DataFrame(data)

# 연령대별로 변환하는 함수 정의
def convert_age_group(age_str):
    if age_str == '20세 미만':
        return '10'
    elif age_str == '85세 이상':
        return '85'
    else:
        return age_str.split('~')[0]

# '연령' 열에 함수 적용하여 연령대별로 변환
df['연령대'] = df['연령'].apply(convert_age_group)

In [31]:
df

,연령,연령대
0,20세 미만,10
1,20~24,20
2,25~29,25
3,30~34,30
4,35~39,35
5,40~44,40
6,45~49,45
7,50~54,50
8,55~59,55
9,60~64,60
